# Clustering Approach - 02 

In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from ydata_profiling import ProfileReport
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import plotly.express as px
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from itertools import combinations
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from factor_analyzer import FactorAnalyzer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import r2_score
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_samples
from collections import Counter
from kmodes.kprototypes import KPrototypes
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


In [2]:
data = pd.read_csv('data (3).csv', index_col='date')
def Basic_Preprocessing(df):
    df.index = pd.to_datetime(df.index)
    df.index = df.index.date  

    
    df.drop(columns=['country','street'],inplace=True)
    df = df[df['price'] != 0]
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: 1 if x!=0 else 0)
    df['waterfront'] =  df['waterfront'].apply(lambda x: 1 if x!=0 else 0)
    df['yr_built'] = df['yr_built'].apply(lambda x: 1 if x > df['yr_built'].median() else 0)
    df['sqft_total'] = df['sqft_basement'] + df['sqft_above']
    df.drop(columns=['sqft_basement','sqft_above'],inplace=True)
    #df.drop(columns=['statezip'],inplace=True)
    df['statezip'] = df['statezip'].apply(lambda x: int(str(x).split()[1]))
    df['month'] = pd.to_datetime(df.index).month
    df['date'] = pd.to_datetime(df.index).day
    
    df.reset_index(drop=True, inplace=True)

    return df 

df = Basic_Preprocessing(data)

X = df.drop(columns=['price'])
Y= df['price']



In [3]:
def imp_cats(data, col, Y):
    df = pd.DataFrame(data[col])
    df['price'] = Y
    dummy = pd.get_dummies(df[col], prefix=f'${col}$', dtype=int)
    df = pd.concat([df, dummy], axis=1)
    df.drop(col, axis=1, inplace=True)

    # Train a Random Forest model
    rf = RandomForestRegressor()
    rf.fit(df.drop('price', axis=1), Y)

    # Get feature importance scores
    feature_importances = rf.feature_importances_

    # Sort features based on their importance scores
    selected_features_indices = np.argsort(feature_importances)[::-1]
    selected_features = df.columns[selected_features_indices][:10]

    return selected_features

## Spliting the data 

In [4]:
X_train , X_test , y_train , y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [5]:
''' ##########  Label encoding 
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
ordinal_encoder.fit(X_train[['city']])

# Transform both training and test sets
X_train['city'] = ordinal_encoder.transform(X_train[['city']])
X_test['city'] = ordinal_encoder.transform(X_test[['city']])

'''

" ##########  Label encoding \nfrom sklearn.preprocessing import OrdinalEncoder\nordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)\nordinal_encoder.fit(X_train[['city']])\n\n# Transform both training and test sets\nX_train['city'] = ordinal_encoder.transform(X_train[['city']])\nX_test['city'] = ordinal_encoder.transform(X_test[['city']])\n\n"

In [16]:


###########  One hot encoding 
combined_df = pd.concat([X_train, X_test])
dummy_cols = ['city']
combined_dummy_df = pd.get_dummies(combined_df[dummy_cols], drop_first=True)
X_train_dummy = combined_dummy_df.iloc[:len(X_train)]
X_test_dummy = combined_dummy_df.iloc[len(X_train):]
X_train_dummy = X_train_dummy.astype(int)
X_test_dummy = X_test_dummy.astype(int)
X_train = X_train.drop(columns=dummy_cols)
X_test = X_test.drop(columns=dummy_cols)
X_train = pd.concat([X_train, X_train_dummy], axis=1)
X_test = pd.concat([X_test, X_test_dummy], axis=1)



In [17]:
'''
#############  Frequency Encoding 
city_frequency = X_train['city'].value_counts(normalize=True)

# Map the frequency values to the corresponding categories in both training and testing datasets
X_train['city'] = X_train['city'].map(city_frequency)
X_test['city'] = X_test['city'].map(city_frequency)

'''

"\n#############  Frequency Encoding \ncity_frequency = X_train['city'].value_counts(normalize=True)\n\n# Map the frequency values to the corresponding categories in both training and testing datasets\nX_train['city'] = X_train['city'].map(city_frequency)\nX_test['city'] = X_test['city'].map(city_frequency)\n\n"

In [18]:
'''
# Merge 'price' from y_train with 'city' in X_train
mean_target_encoding = y_train.groupby(X_train['city']).mean()

# Map the mean target encoding back to the original DataFrame
X_train['city'] = X_train['city'].map(mean_target_encoding)
X_test['city'] = X_test['city'].map(mean_target_encoding)
'''

"\n# Merge 'price' from y_train with 'city' in X_train\nmean_target_encoding = y_train.groupby(X_train['city']).mean()\n\n# Map the mean target encoding back to the original DataFrame\nX_train['city'] = X_train['city'].map(mean_target_encoding)\nX_test['city'] = X_test['city'].map(mean_target_encoding)\n"

In [5]:
## One hot encoding for 10 most frequent cats 
top_10_cities = X_train['city'].value_counts().head(10).index.tolist()

# Filter the DataFrame to include only the top 10 cities
X_train_top_10 = X_train[X_train['city'].isin(top_10_cities)]
X_test_top_10 = X_test[X_test['city'].isin(top_10_cities)]

# Perform one-hot encoding for the top 10 cities
dummy_cols_top_10 = ['city']
combined_dummy_df_top_10 = pd.get_dummies(X_train_top_10[dummy_cols_top_10], drop_first=True)
X_train_dummy_top_10 = combined_dummy_df_top_10.astype(int)

# Apply the same encoding to the test set
X_test_dummy_top_10 = pd.get_dummies(X_test_top_10[dummy_cols_top_10], drop_first=True).astype(int)

# Concatenate the dummy variables with the original DataFrame
X_train = pd.concat([X_train_top_10.drop(columns=dummy_cols_top_10), X_train_dummy_top_10], axis=1)
X_test = pd.concat([X_test_top_10.drop(columns=dummy_cols_top_10), X_test_dummy_top_10], axis=1)

In [6]:
##  by adjusting contamination we can make our model more sensitive to outliers 
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.2)  
clf.fit(X_train)
y_pred = clf.predict(X_train)


In [7]:
sum(y_pred==-1)

547

In [8]:
sum(y_pred==1)

2186

In [9]:
X_train['cluster_lables'] = y_pred

X_train_1 = X_train[X_train['cluster_lables']==-1]
X_train_2 = X_train[X_train['cluster_lables']==1]

indexes_1 = X_train_1.index
indexes_2 = X_train_2.index


y_train_1 = y_train.loc[indexes_1]
y_train_2 = y_train.loc[indexes_2]

# Best model for cluster 1 

In [10]:
x_train_cluster_1 , x_test_cluster_1 , y_train_cluster_1 , y_test_cluster_1 = train_test_split(
                                        X_train_1.drop(columns=['date','month']),y_train_1,random_state=42)

In [11]:
x_train_cluster_1.columns

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'yr_built', 'yr_renovated',
       'statezip', 'sqft_total', 'city_Bellevue', 'city_Federal Way',
       'city_Issaquah', 'city_Kent', 'city_Kirkland', 'city_Redmond',
       'city_Renton', 'city_Sammamish', 'city_Seattle', 'cluster_lables'],
      dtype='object')

In [12]:
vars_to_transform = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'condition', 'sqft_total']

In [13]:
### Min Max scaling for  votes :
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform on training data
x_train_cluster_1 = x_train_cluster_1.copy()  # Create a copy to avoid modifying the original data
x_train_cluster_1[vars_to_transform] = scaler.fit_transform(x_train_cluster_1[vars_to_transform])

# Transform test data using the scaler fitted on training data
x_test_cluster_1 = x_test_cluster_1.copy()  # Create a copy to avoid modifying the original data
x_test_cluster_1[vars_to_transform] = scaler.transform(x_test_cluster_1[vars_to_transform])

In [14]:
y_train_cluster_1 = np.log1p(y_train_cluster_1)
y_test_cluster_1 = np.log1p(y_test_cluster_1)

In [15]:
gb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.5, 0.75, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create GradientBoostingRegressor
gb_reg = GradientBoostingRegressor()

# Grid search
grid_search = GridSearchCV(estimator=gb_reg, param_grid=gb_params,
                           cv=5, scoring='neg_mean_squared_error',
                           n_jobs=-1)
# Fit GridSearchCV
grid_search.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model
best_gb_model_1 = grid_search.best_estimator_



##########################################################################################################
###########################################################################################################
##########   calculate  the adj R squred value   for training 
from sklearn.metrics import r2_score
best_gb_model_1.fit(x_train_cluster_1, y_train_cluster_1)
y_train_pred = best_gb_model_1.predict(x_train_cluster_1)
r_squared = r2_score(y_train_cluster_1, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_1.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########y_test_pred = best_gb_model_1.predict(x_test_cluster_1)
best_gb_model_1.fit(x_test_cluster_1, y_test_cluster_1)
y_test_pred = best_gb_model_1.predict(x_test_cluster_1)
r_squared_test = r2_score(y_test_cluster_1, y_test_pred)
n_test = len(y_test_cluster_1)
p_test = x_test_cluster_1.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)

######################################################################################
# Calculate RMSE for training set
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.842688626342937
Adjusted R-squared (Test): 0.9181144250846109
RMSE (Training): 0.2674955747422373
RMSE (Testing): 0.16410031393040433


In [16]:
#################################   Random Forest   ###########################################
rf_params = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create RandomForestRegressor
rf_reg = RandomForestRegressor()

# Grid search
grid_search_rf = GridSearchCV(estimator=rf_reg, param_grid=rf_params,
                              cv=5, scoring='neg_mean_squared_error',
                              n_jobs=-1)

# Fit GridSearchCV
grid_search_rf.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model
best_rf_model_1 = grid_search_rf.best_estimator_






#########################################################################################################
#########################################################################################################
#################   calculate  the adj R squred value  ####################### #
best_rf_model_1.fit(x_train_cluster_1, y_train_cluster_1)
y_train_pred = best_rf_model_1.predict(x_train_cluster_1)
r_squared = r2_score(y_train_cluster_1, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_1.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_rf_model_1.predict(x_test_cluster_1)
r_squared_test = r2_score(y_test_cluster_1, y_test_pred)
n_test = len(y_test_cluster_1)
p_test = x_test_cluster_1.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)



######################################################################################
# Calculate RMSE for training set
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.8179953002278694
Adjusted R-squared (Test): 0.6022365850777054
RMSE (Training): 0.2877251607454853
RMSE (Testing): 0.3616746987295675


In [16]:
######################################  Ada Boost   ###############################################
from sklearn.ensemble import AdaBoostRegressor
ada_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Create AdaBoostRegressor
ada_reg = AdaBoostRegressor()

# Grid search for AdaBoost
grid_search_ada = GridSearchCV(estimator=ada_reg, param_grid=ada_params,
                                cv=5, scoring='neg_mean_squared_error',
                                n_jobs=-1)

# Fit GridSearchCV for AdaBoost
grid_search_ada.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best AdaBoost model
best_ada_model_1 = grid_search_ada.best_estimator_


############################################################################################################
##########################################################################################################
#################   calculate  the adj R squred value  ########################
best_ada_model_1.fit(x_train_cluster_1, y_train_cluster_1)
y_train_pred = best_ada_model_1.predict(x_train_cluster_1)
r_squared = r2_score(y_train_cluster_1, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_1.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_ada_model_1.predict(x_test_cluster_1)
r_squared_test = r2_score(y_test_cluster_1, y_test_pred)
n_test = len(y_test_cluster_1)
p_test = x_test_cluster_1.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


######################################################################################
# Calculate RMSE for training set
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.6714440087342011
Adjusted R-squared (Test): 0.5347660861893009
RMSE (Training): 0.3506555205771757
RMSE (Testing): 0.3746423075328432


In [17]:
#######################  ElasticNet ###############################
elastic_net_params = {
    'alpha': [0.1, 1.0, 10.0],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Create ElasticNet regressor
elastic_net = ElasticNet()

# Grid search
grid_search_elastic_net = GridSearchCV(estimator=elastic_net, param_grid=elastic_net_params,
                                       cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search_elastic_net.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model
best_elastic_net_model_1 = grid_search_elastic_net.best_estimator_




##########################################################################################################
##########################################################################################################
#################   calculate  the adj R squred value  ########################
best_elastic_net_model_1.fit(x_train_cluster_1, y_train_cluster_1)
y_train_pred =best_elastic_net_model_1.predict(x_train_cluster_1)
r_squared = r2_score(y_train_cluster_1, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_1.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_elastic_net_model_1.predict(x_test_cluster_1)
r_squared_test = r2_score(y_test_cluster_1, y_test_pred)
n_test = len(y_test_cluster_1)
p_test = x_test_cluster_1.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


######################################################################################
# Calculate RMSE for training set
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.4415472750208054
Adjusted R-squared (Test): 0.36583633269163185
RMSE (Training): 0.503999084174611
RMSE (Testing): 0.45667410777513606


In [18]:
####################  Ridge #####################
from sklearn.linear_model import Ridge
ridge_params = {
    'alpha': [0.1, 1.0, 10.0]
}
# Create Ridge regressor
ridge_reg = Ridge()

# Grid search
grid_search_ridge = GridSearchCV(estimator=ridge_reg, param_grid=ridge_params,
                                 cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search_ridge.fit(x_train_cluster_1, y_train_cluster_1)

# Get the best model
best_ridge_model_1 = grid_search_ridge.best_estimator_





###########################################################################################################
###########################################################################################################
#################   calculate  the adj R squred value  ########################
best_ridge_model_1.fit(x_train_cluster_1, y_train_cluster_1)
y_train_pred =best_ridge_model_1.predict(x_train_cluster_1)
r_squared = r2_score(y_train_cluster_1, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_1.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_ridge_model_1.predict(x_test_cluster_1)
r_squared_test = r2_score(y_test_cluster_1, y_test_pred)
n_test = len(y_test_cluster_1)
p_test = x_test_cluster_1.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


######################################################################################
# Calculate RMSE for training set
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_1, y_train_pred))
print("RMSE (Training):", rmse_train)
# Calculate RMSE for testing set
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_1, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.6692298956861347
Adjusted R-squared (Test): 0.580166826973322
RMSE (Training): 0.38788203476359057
RMSE (Testing): 0.37157294642370975


# Best model for cluster 2

In [19]:
x_train_cluster_2 , x_test_cluster_2 , y_train_cluster_2 , y_test_cluster_2 = train_test_split(X_train_2,
                                                                                    y_train_2,random_state=42)

In [20]:
vars_to_transform = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'condition', 'sqft_total']

In [21]:
### Min Max scaling for  votes :
scaler = MinMaxScaler()

# Fit and transform on training data
x_train_cluster_2 = x_train_cluster_2.copy()  # Create a copy to avoid modifying the original data
x_train_cluster_2[vars_to_transform] = scaler.fit_transform(x_train_cluster_2[vars_to_transform])

# Transform test data using the scaler fitted on training data
x_test_cluster_2 = x_test_cluster_2.copy()  # Create a copy to avoid modifying the original data
x_test_cluster_2[vars_to_transform] = scaler.transform(x_test_cluster_2[vars_to_transform])

In [22]:
y_train_cluster_2 = np.log1p(y_train_cluster_2)
y_test_cluster_2 = np.log1p(y_test_cluster_2)

In [23]:
gb_params = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.5, 0.75, 1.0],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create GradientBoostingRegressor
gb_reg = GradientBoostingRegressor()

# Grid search
grid_search = GridSearchCV(estimator=gb_reg, param_grid=gb_params,
                           cv=5, scoring='neg_mean_squared_error',
                           n_jobs=-1)
# Fit GridSearchCV
grid_search.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model
best_gb_model_2 = grid_search.best_estimator_



########################################################################################################
#########################################################################################################
##########   calculate  the adj R squred value   for training 
from sklearn.metrics import r2_score
best_gb_model_2.fit(x_train_cluster_2, y_train_cluster_2)
y_train_pred = best_gb_model_2.predict(x_train_cluster_2)
r_squared = r2_score(y_train_cluster_2, y_train_pred)
n = len(y_train_cluster_2)
p = x_train_cluster_2.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)


###########################################################################################################
##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_gb_model_2.predict(x_test_cluster_2)
r_squared_test = r2_score(y_test_cluster_2, y_test_pred)
n_test = len(y_test_cluster_2)
p_test = x_test_cluster_2.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)



#######################################   RMSE 
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred))
print("RMSE (Training):", rmse_train)
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.8963865676383401
Adjusted R-squared (Test): 0.7440674714034403
RMSE (Training): 0.14485327825114525
RMSE (Testing): 0.22414382090609436


In [ ]:
#################################   Random Forest   ###########################################
rf_params = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create RandomForestRegressor
rf_reg = RandomForestRegressor()

# Grid search
grid_search_rf = GridSearchCV(estimator=rf_reg, param_grid=rf_params,
                              cv=5, scoring='neg_mean_squared_error',
                              n_jobs=-1)

# Fit GridSearchCV
grid_search_rf.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model
best_rf_model_2 = grid_search_rf.best_estimator_






#########################################################################################################
#################   calculate  the adj R squred value  ####################### #
best_rf_model_2.fit(x_train_cluster_2, y_train_cluster_2)
y_train_pred = best_rf_model_2.predict(x_train_cluster_2)
r_squared = r2_score(y_train_cluster_2, y_train_pred)
n = len(y_train_cluster_2)
p = x_train_cluster_2.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)



##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_rf_model_2.predict(x_test_cluster_2)
r_squared_test = r2_score(y_test_cluster_2, y_test_pred)
n_test = len(y_test_cluster_2)
p_test = x_test_cluster_2.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


#######################################   RMSE 
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred))
print("RMSE (Training):", rmse_train)
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred))
print("RMSE (Testing):", rmse_test)

In [ ]:
######################################  Ada Boost   ###############################################
from sklearn.ensemble import AdaBoostRegressor
ada_params = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
}

# Create AdaBoostRegressor
ada_reg = AdaBoostRegressor()

# Grid search for AdaBoost
grid_search_ada = GridSearchCV(estimator=ada_reg, param_grid=ada_params,
                                cv=5, scoring='neg_mean_squared_error',
                                n_jobs=-1)

# Fit GridSearchCV for AdaBoost
grid_search_ada.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best AdaBoost model
best_ada_model_2 = grid_search_ada.best_estimator_



############################################################################################################
#################   calculate  the adj R squred value  ########################
best_ada_model_2.fit(x_train_cluster_2, y_train_cluster_2)
y_train_pred = best_ada_model_2.predict(x_train_cluster_2)
r_squared = r2_score(y_train_cluster_2, y_train_pred)
n = len(y_train_cluster_1)
p = x_train_cluster_2.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)


##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_ada_model_2.predict(x_test_cluster_2)
r_squared_test = r2_score(y_test_cluster_2, y_test_pred)
n_test = len(y_test_cluster_2)
p_test = x_test_cluster_2.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)



###########################################################################################################
## RMSE 
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred))
print("RMSE (Training):", rmse_train)
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred))
print("RMSE (Testing):", rmse_test)

In [24]:
#######################  ElasticNet ###############################
elastic_net_params = {
    'alpha': [0.1, 1.0, 10.0],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Create ElasticNet regressor
elastic_net = ElasticNet()

# Grid search
grid_search_elastic_net = GridSearchCV(estimator=elastic_net, param_grid=elastic_net_params,
                                       cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search_elastic_net.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model
best_elastic_net_model_2 = grid_search_elastic_net.best_estimator_






#########################################################################################################
#################   calculate  the adj R squred value  ########################
best_elastic_net_model_2.fit(x_train_cluster_2, y_train_cluster_2)
y_train_pred =best_elastic_net_model_2.predict(x_train_cluster_2)
r_squared = r2_score(y_train_cluster_2, y_train_pred)
n = len(y_train_cluster_2)
p = x_train_cluster_2.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)

##########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_elastic_net_model_2.predict(x_test_cluster_2)
r_squared_test = r2_score(y_test_cluster_2, y_test_pred)
n_test = len(y_test_cluster_2)
p_test = x_test_cluster_2.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


########################################################################################################
############  RMSE 
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred))
print("RMSE (Training):", rmse_train)
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.2532517318624038
Adjusted R-squared (Test): 0.23361519152328747
RMSE (Training): 0.3888726593757005
RMSE (Testing): 0.3878711438256147


In [25]:
####################  Ridge #####################
from sklearn.linear_model import Ridge
ridge_params = {
    'alpha': [0.1, 1.0, 10.0]
}

# Create Ridge regressor
ridge_reg = Ridge()

# Grid search
grid_search_ridge = GridSearchCV(estimator=ridge_reg, param_grid=ridge_params,
                                 cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit GridSearchCV
grid_search_ridge.fit(x_train_cluster_2, y_train_cluster_2)

# Get the best model
best_ridge_model_2 = grid_search_ridge.best_estimator_





##########################################################################################################
#################   calculate  the adj R squred value  ########################
best_ridge_model_2.fit(x_train_cluster_2, y_train_cluster_2)
y_train_pred =best_ridge_model_2.predict(x_train_cluster_2)
r_squared = r2_score(y_train_cluster_2, y_train_pred)
n = len(y_train_cluster_2)
p = x_train_cluster_2.shape[1]
adjusted_r_squared = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
print("Adjusted R-squared:", adjusted_r_squared)


#########   calculate  the adj R squred value   for testing ##########
y_test_pred = best_ridge_model_2.predict(x_test_cluster_2)
r_squared_test = r2_score(y_test_cluster_2, y_test_pred)
n_test = len(y_test_cluster_2)
p_test = x_test_cluster_2.shape[1]
adjusted_r_squared_test = 1 - (1 - r_squared_test) * (n_test - 1) / (n_test - p_test - 1)
print("Adjusted R-squared (Test):", adjusted_r_squared_test)


########################################################################################################
############  RMSE 
rmse_train = np.sqrt(mean_squared_error(y_train_cluster_2, y_train_pred))
print("RMSE (Training):", rmse_train)
rmse_test = np.sqrt(mean_squared_error(y_test_cluster_2, y_test_pred))
print("RMSE (Testing):", rmse_test)

Adjusted R-squared: 0.6448545471691687
Adjusted R-squared (Test): 0.648988374946696
RMSE (Training): 0.26817817272436
RMSE (Testing): 0.26249726161083237


# Final Evaluation 

In [26]:
clf.fit(X_test)
labels = clf.predict(X_test)
X_test['cluster_lables'] = labels


In [59]:

X_test_1 = X_test[X_test['cluster_lables']==-1]
X_test_1.drop(columns=['month','date'],inplace=True)
X_test_2 =  X_test[X_test['cluster_lables']==1]

# Get indices of cluster 1 and cluster 2 in X_test
indexes_1 = X_test[X_test['cluster_lables'] == -1].index
indexes_2 = X_test[X_test['cluster_lables'] == 1].index

# Filter y_test based on the indices
y_test_1 = y_test.loc[indexes_1]
y_test_2 = y_test.loc[indexes_2]


In [60]:
################  Transformations 

### Min Max scaling for  votes :
scaler = MinMaxScaler()

# Fit and transform on training data
X_test_1 = X_test_1.copy()  # Create a copy to avoid modifying the original data
X_test_1[vars_to_transform] = scaler.fit_transform(X_test_1[vars_to_transform])

# Transform test data using the scaler fitted on training data
X_test_2 = X_test_2.copy()  # Create a copy to avoid modifying the original data
X_test_2[vars_to_transform] = scaler.transform(X_test_2[vars_to_transform])

In [61]:
y_test_1 = np.log1p(y_test_1)
y_test_2 = np.log1p(y_test_2)

In [79]:
best_model_cluster_1 = best_gb_model_1
best_model_cluster_2 = best_elastic_net_model_2

In [80]:
y_test_pred_cluster_1 = best_model_cluster_1.predict(X_test_1)
r_squared_cluster_1 = r2_score(y_test_1, y_test_pred_cluster_1)
n_cluster_1 = len(y_test_1)
p_cluster_1 = X_test_1.shape[1]
adjusted_r_squared_cluster_1 = 1 - (1 - r_squared_cluster_1) * (n_cluster_1 - 1) / (n_cluster_1 - p_cluster_1 - 1)
rmse_cluster_1 = mean_squared_error(y_test_1, y_test_pred_cluster_1, squared=False)

y_test_pred_cluster_2 =  best_model_cluster_2.predict(X_test_2)
r_squared_cluster_2 = r2_score(y_test_2, y_test_pred_cluster_2)
n_cluster_2 = len(y_test_2)
p_cluster_2 = X_test_2.shape[1]
adjusted_r_squared_cluster_2 = 1 - (1 - r_squared_cluster_2) * (n_cluster_2 - 1) / (n_cluster_2 - p_cluster_2 - 1)
rmse_cluster_2 = mean_squared_error(y_test_2, y_test_pred_cluster_2, squared=False)


In [81]:
# Combine predictions for both clusters
y_test_pred = np.concatenate((y_test_pred_cluster_1, y_test_pred_cluster_2))

# Combine actual target values for both clusters
y_test_combined = pd.concat([y_test_1, y_test_2])

# Calculate R-squared for the combined test set
r_squared_test_combined = r2_score(y_test_combined, y_test_pred)

# Calculate mean squared error (MSE) for the combined test set
mse_test_combined = mean_squared_error(y_test_combined, y_test_pred)

# Calculate root mean squared error (RMSE) for the combined test set
rmse_test_combined = np.sqrt(mse_test_combined)

# Print the final test R-squared score, MSE, and RMSE
print("Final Test R-squared:", r_squared_test_combined)
print("Final Test MSE:", mse_test_combined)
print("Final Test RMSE:", rmse_test_combined)

Final Test R-squared: 0.34746128545556887
Final Test MSE: 0.17965955304646353
Final Test RMSE: 0.4238626582354991
